In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('/kaggle/input/deep-learning-2-0/data.csv')

In [3]:
df.head()

,Date,Time,P1,P2,P3,P4,P5,P6,P7,P8,P9
0,15-10-2011,39:55.0,-0.0469,-0.0385,0.00542,0.01121,38,52,0,0,0
1,15-10-2011,39:55.6,-0.0469,-0.0385,0.00542,0.01121,38,52,0,0,0
2,15-10-2011,39:57.1,-0.0469,-0.0385,0.00542,0.01121,38,52,0,0,0
3,15-10-2011,39:58.6,-0.0469,-0.0385,0.00542,0.01121,38,52,0,0,0
4,15-10-2011,40:02.2,-0.0469,-0.0385,0.00542,0.01121,38,52,0,0,0


In [4]:
# Assuming 'Date' and 'Time' columns exist in your DataFrame df
datetime = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %M:%S.%f')

# Drop 'Date' and 'Time' columns as they are no longer needed
df.drop(['Date', 'Time'], axis=1, inplace=True)

In [5]:
df.columns

Index(['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9'], dtype='object')

In [6]:
# Normalize the dataset using Min-Max scaling
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

In [7]:
df['datetime']=datetime

In [8]:
df.columns

Index(['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'datetime'], dtype='object')

In [9]:
df['datetime'].head()

0   2011-10-15 00:39:55.000
1   2011-10-15 00:39:55.600
2   2011-10-15 00:39:57.100
3   2011-10-15 00:39:58.600
4   2011-10-15 00:40:02.200
Name: datetime, dtype: datetime64[ns]

In [ ]:
# Define a function to create sequences for LSTM training
def create_sequences(dataset, target_column_index, n_steps):
    X, y = [], []
    for i in range(len(dataset) - n_steps):
        seq_x = dataset[i:i + n_steps, :]
        seq_y = dataset[i + n_steps, target_column_index]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# Set the number of time steps (input sequence length)
n_steps = 10

In [ ]:
# Set the index of the target parameter in your dataset
target_column_index = 0  # You need to adjust this based on your dataset

In [ ]:
# Create sequences for training
X, y = create_sequences(df_scaled, target_column_index, n_steps)

In [ ]:
# Split the data into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(X))

In [ ]:
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [ ]:
# Define and compile the LSTM model with three Dense layers
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(n_steps, df_scaled.shape[1])))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=25, activation='relu'))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=16, verbose=1)

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
# Inverse transform the predictions and actual values to the original scale
y_pred_inv = scaler.inverse_transform(np.concatenate((np.zeros((len(y_pred), df_scaled.shape[1]-1)), y_pred), axis=1))[:, target_column_index]
y_test_inv = scaler.inverse_transform(np.concatenate((np.zeros((len(y_test), df_scaled.shape[1]-1)), y_test.reshape(-1, 1)), axis=1))[:, target_column_index]

In [ ]:
mse = mean_squared_error(y_test_inv, y_pred_inv)
print(f'Mean Squared Error: {mse}')

# Plot the results
plt.plot(y_test_inv, label='Actual')
plt.plot(y_pred_inv, label='Predicted')
plt.legend()
plt.show()